# Load Train Set

features (3681) | labels (1)

In [1]:
# import pandas as pd
import cudf
import cupy
import pandas as pd
import numpy as np

# df = pd.read_feather("dts/np_dataset_train.ftr")
df = pd.read_csv("dts/np_dataset_train.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 1.1 GB


## train/test Split

In [2]:
from sklearn.model_selection import train_test_split

rs = 2

train, test = train_test_split(df, test_size=0.2, random_state=rs)
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 31392 to 23720
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 899.4 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 2727 to 32354
Columns: 3684 entries, Unnamed: 0 to Label
dtypes: bool(1), float64(3682), int64(1)
memory usage: 224.9 MB


## Classifier definition

In [3]:
# from sklearn import svm
from cuml.svm import SVC

clf = SVC(random_state = rs)


## Sending Data to GPU

In [4]:
x = cudf.DataFrame(train.iloc[:, :-1])
# y = (train.iloc[:, -1])

y = cupy.array(train.iloc[:, -1])

print(x.shape)
print(y.shape)

(32000, 3683)
(32000,)


## Model Fitting

In [5]:
clf.fit(x, y)

SVC()

## Testing

In [6]:
y_pred = clf.predict(test.iloc[:,:-1])

## Evaluating

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
roc = roc_auc_score(test.iloc[:, -1], y_pred)
pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)


print("Metrics: \n Acc: {acc:.2f}, ROC: {roc:.2f}\n PRE: {pre:.2f}, REC: {rec:.2f}".format(acc=accuracy,roc=roc,pre=pre,rec=rec))


Metrics: 
 Acc: 1.00, ROC: 1.00
 PRE: 1.00, REC: 1.00


## SC graph

In [10]:
accs = []
rocs = []
pres = []
recs = []


for rs in range(100):
    train, test = train_test_split(df, test_size=0.2, random_state=rs)

    clf = SVC(random_state=rs)

    x = cudf.DataFrame(train.iloc[:, :-1])
    y = cupy.array(train.iloc[:, -1])

    clf.fit(x, y)
    y_pred = clf.predict(test.iloc[:, :-1])

    accuracy = accuracy_score(y_true=test.iloc[:, -1], y_pred=y_pred)
    accs.append(accuracy)
    roc = roc_auc_score(test.iloc[:, -1], y_pred)
    rocs.append(roc)
    pre = precision_score(y_true=test.iloc[:, -1], y_pred=y_pred)
    pres.append(pre)
    rec = recall_score(y_true=test.iloc[:, -1], y_pred=y_pred)
    recs.append(rec)

    print("{i}/100".format(i=rs+1))

In [19]:


# import matplotlib.pyplot as plt


# plt.stairs(accs)

# npAccs = np.array(accs)
# npRocs = np.array(rocs)
# npPres = np.array(pres)
# npRecs = np.array(recs)

def min_max(name,array):
    a = np.array(array)
    print("{name}: ({min:.6f},{max:.1f}|{mean:.6f})".format(
        name=name, min=np.min(a), max=np.max(a), mean=np.mean(a)))

min_max("Acc",accs)
min_max("Roc", rocs)
min_max("Pre", pres)
min_max("Rec", recs)


Acc: (0.998750,1.0|0.999829)
Roc: (0.998751,1.0|0.999829)
Pre: (0.997504,1.0|0.999832)
Rec: (0.997532,1.0|0.999826)
